In [1]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)

In [2]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""
import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from scipy import ndimage
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.stats import median_absolute_deviation
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib


import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat

calc_emergence =False
debris_only=False

min_dhdt_perc = 75

debug=False
extra_layers=True

csv_ending = '_mb_bins.csv'
# outdir_csv = debris_prms.mb_binned_fp
outdir_csv = ('/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/debris_global/' + 
              '../output/mb_bins_all/csv/')
outdir_csv = outdir_csv + debris_prms.roi + '/'
if not os.path.exists(outdir_csv):
    os.makedirs(outdir_csv)
    
# dhdt_vel_fns_fp = debris_prms.dhdt_vel_fns_fp
dhdt_vel_fns_fp = ('/Users/davidrounce/Documents/Dave_Rounce/DebrisGlaciers_WG/Melt_Intercomparison/debris_global/' + 
                   '../output/dhdt_vel_fns_all/')
if not os.path.exists(dhdt_vel_fns_fp):
    os.makedirs(dhdt_vel_fns_fp)


In [3]:
if debris_only:
    # Debris cover extent shapefile with statistics
    dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
    dc_shp = dc_shp.sort_values(by=['RGIId'])

    # Subset by percent debris-covered or debris-covered area
    dc_shp_subset = dc_shp[((dc_shp['DC_Area__1'] > debris_prms.dc_percarea_threshold) | 
                            (dc_shp['DC_Area_v2'] / 1e6 > debris_prms.dc_area_threshold))
                            & (dc_shp['Area'] > debris_prms.min_glac_area)].copy()
    dc_shp_subset.reset_index(inplace=True, drop=True)
    dc_shp_subset['CenLon_360'] = dc_shp_subset['CenLon']
    dc_shp_subset.loc[dc_shp_subset['CenLon_360'] < 0, 'CenLon_360'] = (
        360 + dc_shp_subset.loc[dc_shp_subset['CenLon_360'] < 0, 'CenLon_360'])

    rgiid_list = [x.split('-')[1] for x in dc_shp_subset['RGIId'].values]
    main_glac_rgi_subset = debris_prms.selectglaciersrgitable(rgiid_list)

else:
    # All glaciers
    main_glac_rgi_subset = debris_prms.selectglaciersrgitable(rgi_regionsO1=debris_prms.roi_rgidict[debris_prms.roi],
                                                              rgi_regionsO2='all', rgi_glac_number='all')
    
    # Debris cover extent shapefile with statistics
    dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
    dc_shp = dc_shp.sort_values(by=['RGIId'])
    dc_shp.reset_index(inplace=True, drop=True)

# dc_shp
main_glac_rgi_subset

All glaciers within region(s) 13 are included in this model run.
All glaciers within region(s) 14 are included in this model run.
All glaciers within region(s) 15 are included in this model run.
This study is focusing on 95536 glaciers in region [13, 14, 15]


,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,Slope,Aspect,Lmax,Form,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float
GlacNo,,,,,,,,,,,,,,,,,,,,
0,0,RGI60-13.00001,79.4939,34.4713,13,5,0.432,5678,5928,5763,22.0,312,683,0,0,9,20000810,1,13.00001,13.00001
1,1,RGI60-13.00002,77.9513,35.5452,13,5,0.367,5889,6217,6051,25.4,195,1307,0,0,9,20020802,2,13.00002,13.00002
2,2,RGI60-13.00003,77.9295,35.5188,13,5,0.070,5646,5849,5756,27.3,318,426,0,0,9,20020802,3,13.00003,13.00003
3,3,RGI60-13.00004,77.9237,35.5252,13,5,0.255,5698,5850,5756,16.9,39,605,0,0,9,20020802,4,13.00004,13.00004
4,4,RGI60-13.00005,77.9141,35.5309,13,5,0.261,5644,5809,5745,19.0,8,389,0,0,9,20020802,5,13.00005,13.00005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95531,13114,RGI60-15.13115,81.9770,30.3370,15,1,1.356,5480,5852,5637,12.6,352,1856,0,0,9,20090918,13115,15.13115,15.13115
95532,13115,RGI60-15.13116,81.9910,30.3390,15,1,7.427,5266,6232,5633,8.9,330,5779,0,0,9,20090918,13116,15.13116,15.13116
95533,13116,RGI60-15.13117,81.3550,30.4400,15,1,6.788,5696,6147,5994,6.3,96,5519,0,0,9,20070912,13117,15.13117,15.13117


In [4]:
dhdt_vel_fns_fn = debris_prms.dhdt_vel_fns_fn.replace('XXXX',debris_prms.roi)
if os.path.exists(dhdt_vel_fns_fp + dhdt_vel_fns_fn):
    dhdt_vel_fns_df = pd.read_csv(dhdt_vel_fns_fp + dhdt_vel_fns_fn)
else:
    dhdt_vel_fns_df = pd.DataFrame(np.zeros((main_glac_rgi_subset.shape[0], 3)),
                                   columns=['RGIId', 'dhdt_fullfn', 'vel_fullfn'])
    dhdt_vel_fns_df['RGIId'] = main_glac_rgi_subset['RGIId']

In [5]:
# np.where(main_glac_rgi_subset.rgino_str == '11.03005')

In [6]:
# skip existing
rgiid_existing = []
for i in os.listdir(outdir_csv):
    if i.endswith('.csv'):
        rgiid_existing.append(i.split('_')[0])
print('existing:', len(rgiid_existing))

existing: 0


In [ ]:
# ===== PROCESS EACH GLACIER =====
dc_rgiids = dc_shp.RGIId.tolist()
for nglac, glac_idx in enumerate(main_glac_rgi_subset.index.values):
# for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[0]]):
# for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[120]]): # Miage
# for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[2307]]): # Ngozumpa

    glac_str = main_glac_rgi_subset.loc[glac_idx,'rgino_str']
    rgiid = main_glac_rgi_subset.loc[glac_idx,'RGIId']
    region = glac_str.split('.')[0]

    if int(region) < 10:
        glac_str_noleadzero = str(int(glac_str.split('.')[0])) + '.' + glac_str.split('.')[1]
    else:
        glac_str_noleadzero = glac_str
        
    if glac_str_noleadzero not in rgiid_existing:

        print(nglac, glac_idx, rgiid)

        # Create glacier feature from ice thickness raster
        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
        
        if os.path.exists(thick_dir + thick_fn):
        
            gf = create_glacfeat(thick_dir, thick_fn)

            if rgiid in dc_rgiids:
                # Debris shape layer processing
                dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                                  str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
                if os.path.exists(dc_shp_proj_fn) == False:
                    dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
                    dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                    dc_shp_single = dc_shp_single.reset_index()
                    dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                    dc_shp_proj.to_file(dc_shp_proj_fn)
                dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
                dc_shp_lyr = dc_shp_ds.GetLayer()
            else:
                dc_shp_lyr = None

            # ==== CHECK IF TIF HAS DHDT DATA OVER THE GLACIER =====
            mb_fullfns = []
            find_mb = True
            dhdt_fn_wglacier = None
            for mb_fp in debris_prms.mb_fp_list_roi[debris_prms.roi]:
                if find_mb:
                    for i in os.listdir(mb_fp):
                        if i.endswith('.tif'):
                            mb_fullfns.append(mb_fp + i)
                    tif_count = 0
                    while find_mb and tif_count < len(mb_fullfns):
                        dhdt_fn = mb_fullfns[tif_count]
                        if debug:
                            print(tif_count, dhdt_fn.split('/')[-1])

                        # Add the filenames
                        fn_dict = OrderedDict()
                        # DEM
                        z1_fp = debris_prms.oggm_fp + 'dems/RGI60-' + str(region.zfill(2)) + '/'
                        z1_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_dem.tif'
                        fn_dict['z1'] = z1_fp + z1_fn
                        # Ice thickness
                        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
                        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
                        fn_dict['ice_thick'] = thick_dir + thick_fn
                        # dh/dt
                        fn_dict['dhdt'] = dhdt_fn
                        # ===== PROCESS THE DATA =====
                        #Expand extent to include buffered region around glacier polygon
                        warp_extent = geolib.pad_extent(gf.glac_geom_extent, width=debris_prms.buff_dist)
                        #Warp everything to common res/extent/proj
                        z1_gt = gdal.Open(fn_dict['z1']).GetGeoTransform()
                        z1_res = np.min([z1_gt[1], -z1_gt[5]])
                        ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, extent=warp_extent, 
                                                           t_srs=gf.aea_srs, verbose=False, r='cubic')
                        ds_dict = dict(zip(fn_dict.keys(), ds_list))
                        gf.ds_dict = ds_dict

                        if 'z1' in ds_dict:
                            #This is False over glacier polygon surface, True elsewhere - can be applied directly
                            glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1'])
                            glac_geom_mask_copy = glac_geom_mask.copy()
                            gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)
                            if rgiid in dc_rgiids:
                                # Debris cover
                                dc_shp_lyr_mask = geolib.lyr2mask(dc_shp_lyr, ds_dict['ice_thick'])
                                gf.dc_mask = np.ma.mask_or(dc_shp_lyr_mask, glac_geom_mask)
                            if 'dhdt' in ds_dict:
                                gf.dhdt = np.ma.array(iolib.ds_getma(ds_dict['dhdt']), mask=glac_geom_mask_copy)
                                gf.dhdt.mask = np.ma.mask_or(
                                    glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.dhdt.data, 
                                                                                     np.isnan(gf.dhdt.data))))
                                if rgiid in dc_rgiids:
                                    gf.dc_dhdt = np.ma.array(iolib.ds_getma(ds_dict['dhdt']), mask=glac_geom_mask_copy)
                                    gf.dc_dhdt.mask = gf.dc_mask
                                    gf.dc_area = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=gf.dc_mask)
                                # Count dhdt pixels
                                dhdt_pixels = len(gf.dhdt.nonzero()[0])
                                if dhdt_pixels / gf.z1.count() * 100 > min_dhdt_perc:
                                    dhdt_fn_wglacier = dhdt_fn
                                    find_mb = False
                                    if debug:
                                        print('\n# z1 pixels:', gf.z1.count())
                                        print('# dhdt_pixels:', dhdt_pixels)
                                        var_full2plot = gf.dhdt.copy()
                                        clim = malib.calcperc(var_full2plot, (2,98))
                                        plot_array(var_full2plot, clim, [glac_str + ' dhdt'], 'inferno', 'dhdt (m/yr)', 
                                                   close_fig=False)
                        # Loop over layers        
                        tif_count += 1

            # ==== CHECK IF VELOCITY DATA OVER THE GLACIER =====
            vel_fullfns = []
            find_vel = True
            vx_fn_wglacier = None
            if find_vel and dhdt_fn_wglacier is not None:
                vx_fns = debris_prms.vx_dir_dict_list[debris_prms.roi]
                tif_count = 0
                while find_vel and tif_count < len(vx_fns):
                    vx_fn = vx_fns[tif_count]

                    if debug:
                        print(tif_count, vx_fn.split('/')[-1])

                    # Add the filenames
                    # Velocity
                    fn_dict['vx'] = vx_fn
                    fn_dict['vy'] = vx_fn.replace('_vx', '_vy')
                    # ===== PROCESS THE DATA =====
                    ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, extent=warp_extent, 
                                                       t_srs=gf.aea_srs, verbose=False, r='cubic')
                    ds_dict = dict(zip(fn_dict.keys(), ds_list))
                    gf.ds_dict = ds_dict
                    if 'vx' in ds_dict and 'vy' in ds_dict:
                        #Load surface velocity maps
                        gf.vx = np.ma.array(iolib.ds_getma(ds_dict['vx']), mask=glac_geom_mask)
                        gf.vy = np.ma.array(iolib.ds_getma(ds_dict['vy']), mask=glac_geom_mask)
                        gf.vm = np.ma.sqrt(gf.vx**2 + gf.vy**2)
                        # Count velocity pixels
                        vel_pixels = len(gf.vm.nonzero()[0])
                        if debug:
                                print('\n# z1 pixels:', gf.z1.count())
                                print('# vel_pixels:', vel_pixels)
                                var_full2plot = gf.vm.copy()
                                clim = malib.calcperc(var_full2plot, (2,98))
                                plot_array(var_full2plot, clim, [glac_str + ' velocity'], 'inferno', 'vm (m/yr)', 
                                           close_fig=False)
                        if vel_pixels / gf.z1.count() * 100 > min_dhdt_perc:
                            vx_fn_wglacier = vx_fn
                            find_vel = False

                    # Loop over layers        
                    tif_count += 1


            # ===== Add layers =====
            if dhdt_fn_wglacier is not None and vx_fn_wglacier is not None:
                gf.add_layers(dc_shp_lyr, gf_add_dhdt=True, dhdt_fn=dhdt_fn_wglacier, gf_add_vel=True, 
                              vx_fn=vx_fn_wglacier, gf_add_ts=False, gf_add_slope_aspect=True, gf_add_ts_info=False, 
                              calc_emergence=calc_emergence, debug_emergence=False)
                # Save dhdt and vel filenames
                dhdt_vel_fns_df.loc[glac_idx,:] = [rgiid, dhdt_fn_wglacier, vx_fn_wglacier]

                # ===== PLOTS =====
#                 plot_layers = True
                plot_layers = False
                if plot_layers:
                    # DEM
                    var_full2plot = gf.z1.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
                    # Elevation change
                    var_full2plot = gf.dhdt.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' dhdt'], 'inferno', 'dhdt (m/yr)', close_fig=False)
                    # Velocity
                    var_full2plot = gf.vm.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' velocity'], 'inferno', 'vel (m/yr)', close_fig=False)
                    # Emergence velocity
                    if gf.emvel is not None:
                        var_full2plot = gf.emvel.copy()
                        clim = malib.calcperc(var_full2plot, (2,98))
                        plot_array(var_full2plot, clim, [glac_str + ' emvel'], 'inferno', 'emvel (m/yr)', close_fig=False)
                    # Surface temperature
                    if gf.ts is not None:
                        var_full2plot = gf.ts.copy()
                        clim = malib.calcperc(var_full2plot, (2,98))
                        plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)

                # Bin data
                outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                # Export binned data
                if int(gf.feat_fn.split('.')[0]) < 10:
                    outbins_fullfn = os.path.join(outdir_csv, gf.feat_fn[0:7] + csv_ending)
                else:
                    outbins_fullfn = os.path.join(outdir_csv, gf.feat_fn[0:8] + csv_ending)
                outbins_df.loc[:,:] = np.nan_to_num(outbins_df.loc[:,:],0)
                outbins_df.to_csv(outbins_fullfn, index=False)

# Save updated filenames
dhdt_vel_fns_df.to_csv(dhdt_vel_fns_fp + dhdt_vel_fns_fn, index=False)

0 0 RGI60-13.00001
1 1 RGI60-13.00002


/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


2 2 RGI60-13.00003
3 3 RGI60-13.00004
4 4 RGI60-13.00005
5 5 RGI60-13.00006
6 6 RGI60-13.00007
7 7 RGI60-13.00008
8 8 RGI60-13.00009
9 9 RGI60-13.00010
10 10 RGI60-13.00011
11 11 RGI60-13.00012
12 12 RGI60-13.00013
13 13 RGI60-13.00014
14 14 RGI60-13.00015
15 15 RGI60-13.00016
16 16 RGI60-13.00017
17 17 RGI60-13.00018
18 18 RGI60-13.00019
19 19 RGI60-13.00020
20 20 RGI60-13.00021
21 21 RGI60-13.00022
22 22 RGI60-13.00023
23 23 RGI60-13.00024
24 24 RGI60-13.00025
25 25 RGI60-13.00026
26 26 RGI60-13.00027
27 27 RGI60-13.00028
28 28 RGI60-13.00029
29 29 RGI60-13.00030
30 30 RGI60-13.00031
31 31 RGI60-13.00032
32 32 RGI60-13.00033
33 33 RGI60-13.00034
34 34 RGI60-13.00035
35 35 RGI60-13.00036
36 36 RGI60-13.00037
37 37 RGI60-13.00038
38 38 RGI60-13.00039
39 39 RGI60-13.00040
40 40 RGI60-13.00041
41 41 RGI60-13.00042
42 42 RGI60-13.00043
43 43 RGI60-13.00044
44 44 RGI60-13.00045
45 45 RGI60-13.00046
46 46 RGI60-13.00047
47 47 RGI60-13.00048
48 48 RGI60-13.00049
49 49 RGI60-13.00050
50 50 RG

368 368 RGI60-13.00369
369 369 RGI60-13.00370
370 370 RGI60-13.00371
371 371 RGI60-13.00372
372 372 RGI60-13.00373
373 373 RGI60-13.00374
374 374 RGI60-13.00375
375 375 RGI60-13.00376
376 376 RGI60-13.00377
377 377 RGI60-13.00378
378 378 RGI60-13.00379
379 379 RGI60-13.00380
380 380 RGI60-13.00381
381 381 RGI60-13.00382
382 382 RGI60-13.00383
383 383 RGI60-13.00384
384 384 RGI60-13.00385
385 385 RGI60-13.00386
386 386 RGI60-13.00387
387 387 RGI60-13.00388
388 388 RGI60-13.00389
389 389 RGI60-13.00390
390 390 RGI60-13.00391
391 391 RGI60-13.00392
392 392 RGI60-13.00393
393 393 RGI60-13.00394
394 394 RGI60-13.00395
395 395 RGI60-13.00396
396 396 RGI60-13.00397
397 397 RGI60-13.00398
398 398 RGI60-13.00399
399 399 RGI60-13.00400
400 400 RGI60-13.00401
401 401 RGI60-13.00402
402 402 RGI60-13.00403
403 403 RGI60-13.00404
404 404 RGI60-13.00405
405 405 RGI60-13.00406
406 406 RGI60-13.00407
407 407 RGI60-13.00408
408 408 RGI60-13.00409
409 409 RGI60-13.00410
410 410 RGI60-13.00411
411 411 RGI

725 725 RGI60-13.00726
726 726 RGI60-13.00727
727 727 RGI60-13.00728
728 728 RGI60-13.00729
729 729 RGI60-13.00730
730 730 RGI60-13.00731
731 731 RGI60-13.00732
732 732 RGI60-13.00733
733 733 RGI60-13.00734
734 734 RGI60-13.00735
735 735 RGI60-13.00736
736 736 RGI60-13.00737
737 737 RGI60-13.00738
738 738 RGI60-13.00739
739 739 RGI60-13.00740
740 740 RGI60-13.00741
741 741 RGI60-13.00742
742 742 RGI60-13.00743
743 743 RGI60-13.00744
744 744 RGI60-13.00745
745 745 RGI60-13.00746
746 746 RGI60-13.00747
747 747 RGI60-13.00748
748 748 RGI60-13.00749
749 749 RGI60-13.00750
750 750 RGI60-13.00751
751 751 RGI60-13.00752
752 752 RGI60-13.00753
753 753 RGI60-13.00754
754 754 RGI60-13.00755
755 755 RGI60-13.00756
756 756 RGI60-13.00757
757 757 RGI60-13.00758
758 758 RGI60-13.00759
759 759 RGI60-13.00760
760 760 RGI60-13.00761
761 761 RGI60-13.00762
762 762 RGI60-13.00763
763 763 RGI60-13.00764
764 764 RGI60-13.00765
765 765 RGI60-13.00766
766 766 RGI60-13.00767
767 767 RGI60-13.00768
768 768 RGI

...

13587 13587 RGI60-13.13588
13588 13588 RGI60-13.13589
13589 13589 RGI60-13.13590
13590 13590 RGI60-13.13591
13591 13591 RGI60-13.13592
13592 13592 RGI60-13.13593
13593 13593 RGI60-13.13594
13594 13594 RGI60-13.13595
13595 13595 RGI60-13.13596
13596 13596 RGI60-13.13597
13597 13597 RGI60-13.13598
13598 13598 RGI60-13.13599
13599 13599 RGI60-13.13600
13600 13600 RGI60-13.13601
13601 13601 RGI60-13.13602
13602 13602 RGI60-13.13603
13603 13603 RGI60-13.13604
13604 13604 RGI60-13.13605
13605 13605 RGI60-13.13606
13606 13606 RGI60-13.13607
13607 13607 RGI60-13.13608
13608 13608 RGI60-13.13609
13609 13609 RGI60-13.13610
13610 13610 RGI60-13.13611
13611 13611 RGI60-13.13612
13612 13612 RGI60-13.13613
13613 13613 RGI60-13.13614
13614 13614 RGI60-13.13615
13615 13615 RGI60-13.13616
13616 13616 RGI60-13.13617
13617 13617 RGI60-13.13618
13618 13618 RGI60-13.13619
13619 13619 RGI60-13.13620
13620 13620 RGI60-13.13621
13621 13621 RGI60-13.13622
13622 13622 RGI60-13.13623
13623 13623 RGI60-13.13624
1

13891 13891 RGI60-13.13892
13892 13892 RGI60-13.13893
13893 13893 RGI60-13.13894
13894 13894 RGI60-13.13895
13895 13895 RGI60-13.13896
13896 13896 RGI60-13.13897
13897 13897 RGI60-13.13898
13898 13898 RGI60-13.13899
13899 13899 RGI60-13.13900
13900 13900 RGI60-13.13901
13901 13901 RGI60-13.13902
13902 13902 RGI60-13.13903
13903 13903 RGI60-13.13904
13904 13904 RGI60-13.13905
13905 13905 RGI60-13.13906
13906 13906 RGI60-13.13907
13907 13907 RGI60-13.13908
13908 13908 RGI60-13.13909
13909 13909 RGI60-13.13910
13910 13910 RGI60-13.13911
13911 13911 RGI60-13.13912
13912 13912 RGI60-13.13913
13913 13913 RGI60-13.13914
13914 13914 RGI60-13.13915
13915 13915 RGI60-13.13916
13916 13916 RGI60-13.13917
13917 13917 RGI60-13.13918
13918 13918 RGI60-13.13919
13919 13919 RGI60-13.13920
13920 13920 RGI60-13.13921
13921 13921 RGI60-13.13922
13922 13922 RGI60-13.13923
13923 13923 RGI60-13.13924
13924 13924 RGI60-13.13925
13925 13925 RGI60-13.13926
13926 13926 RGI60-13.13927
13927 13927 RGI60-13.13928
1

14196 14196 RGI60-13.14197
14197 14197 RGI60-13.14198
14198 14198 RGI60-13.14199
14199 14199 RGI60-13.14200
14200 14200 RGI60-13.14201
14201 14201 RGI60-13.14202
14202 14202 RGI60-13.14203
14203 14203 RGI60-13.14204
14204 14204 RGI60-13.14205
14205 14205 RGI60-13.14206
14206 14206 RGI60-13.14207
14207 14207 RGI60-13.14208
14208 14208 RGI60-13.14209
14209 14209 RGI60-13.14210
14210 14210 RGI60-13.14211
14211 14211 RGI60-13.14212
14212 14212 RGI60-13.14213
14213 14213 RGI60-13.14214
14214 14214 RGI60-13.14215
14215 14215 RGI60-13.14216
14216 14216 RGI60-13.14217
14217 14217 RGI60-13.14218
14218 14218 RGI60-13.14219
14219 14219 RGI60-13.14220
14220 14220 RGI60-13.14221
14221 14221 RGI60-13.14222
14222 14222 RGI60-13.14223
14223 14223 RGI60-13.14224
14224 14224 RGI60-13.14225
14225 14225 RGI60-13.14226
14226 14226 RGI60-13.14227
14227 14227 RGI60-13.14228
14228 14228 RGI60-13.14229
14229 14229 RGI60-13.14230
14230 14230 RGI60-13.14231
14231 14231 RGI60-13.14232
14232 14232 RGI60-13.14233
1

14501 14501 RGI60-13.14502
14502 14502 RGI60-13.14503
14503 14503 RGI60-13.14504
14504 14504 RGI60-13.14505
14505 14505 RGI60-13.14506
14506 14506 RGI60-13.14507
14507 14507 RGI60-13.14508
14508 14508 RGI60-13.14509
14509 14509 RGI60-13.14510
14510 14510 RGI60-13.14511
14511 14511 RGI60-13.14512
14512 14512 RGI60-13.14513
14513 14513 RGI60-13.14514
14514 14514 RGI60-13.14515
14515 14515 RGI60-13.14516
14516 14516 RGI60-13.14517
14517 14517 RGI60-13.14518
14518 14518 RGI60-13.14519
14519 14519 RGI60-13.14520
14520 14520 RGI60-13.14521
14521 14521 RGI60-13.14522
14522 14522 RGI60-13.14523
14523 14523 RGI60-13.14524
14524 14524 RGI60-13.14525
14525 14525 RGI60-13.14526
14526 14526 RGI60-13.14527
14527 14527 RGI60-13.14528
14528 14528 RGI60-13.14529
14529 14529 RGI60-13.14530
14530 14530 RGI60-13.14531
14531 14531 RGI60-13.14532
14532 14532 RGI60-13.14533
14533 14533 RGI60-13.14534
14534 14534 RGI60-13.14535
14535 14535 RGI60-13.14536
14536 14536 RGI60-13.14537
14537 14537 RGI60-13.14538
1

14805 14805 RGI60-13.14806
14806 14806 RGI60-13.14807
14807 14807 RGI60-13.14808
14808 14808 RGI60-13.14809
14809 14809 RGI60-13.14810
14810 14810 RGI60-13.14811
14811 14811 RGI60-13.14812
14812 14812 RGI60-13.14813
14813 14813 RGI60-13.14814
14814 14814 RGI60-13.14815
14815 14815 RGI60-13.14816
14816 14816 RGI60-13.14817
14817 14817 RGI60-13.14818
14818 14818 RGI60-13.14819
14819 14819 RGI60-13.14820
14820 14820 RGI60-13.14821
14821 14821 RGI60-13.14822
14822 14822 RGI60-13.14823
14823 14823 RGI60-13.14824
14824 14824 RGI60-13.14825
14825 14825 RGI60-13.14826
14826 14826 RGI60-13.14827
14827 14827 RGI60-13.14828
14828 14828 RGI60-13.14829
14829 14829 RGI60-13.14830
14830 14830 RGI60-13.14831
14831 14831 RGI60-13.14832
14832 14832 RGI60-13.14833
14833 14833 RGI60-13.14834
14834 14834 RGI60-13.14835
14835 14835 RGI60-13.14836
14836 14836 RGI60-13.14837
14837 14837 RGI60-13.14838
14838 14838 RGI60-13.14839
14839 14839 RGI60-13.14840
14840 14840 RGI60-13.14841
14841 14841 RGI60-13.14842
1

In [ ]:
print('\n\nDONE!\n\n')

In [ ]:
# skip existing
rgiid_existing = []
for i in os.listdir(outdir_csv):
    if i.endswith('.csv'):
        rgiid_existing.append(i.split('_')[0])
main_glac_rgi_processed = debris_prms.selectglaciersrgitable(rgiid_existing)
print('Processed ' + str(len(rgiid_existing)) + ' of ' + str(len(main_glac_rgi_subset)) + ' glaciers')
area_total = main_glac_rgi_subset.Area.sum()
area_processed = main_glac_rgi_processed.Area.sum()
print('  Area: ' + str(np.round(area_processed,1)) + ' km2 (' + str(np.round(area_processed / area_total * 100,1)) + 
      '%) of ' +  str(np.round(area_total,1)) + ' km2')

In [10]:
# ===== SHEAN ESTIMATE OF FLUX DIVERGENCE QUICKLY ======
#                 if gf.H is not None:
#                     #Compute flux
#                     gf.Q = gf.H * debris_prms.v_col_f * np.array([gf.vx, gf.vy])
#                     #Note: np.gradient returns derivatives relative to axis number, so (y, x) in this case
#                     #Want x-derivative of x component
#                     gf.divQ = np.gradient(gf.Q[0])[1] + np.gradient(gf.Q[1])[0]
# #                     gf.divQ = gf.H*(np.gradient(v_col_f*gf.vx)[1] + np.gradient(v_col_f*gf.vy)[0]) \
# #                             + v_col_f*gf.vx*(np.gradient(gf.H)[1]) + v_col_f*gf.vy*(np.gradient(gf.H)[0])
#                     #Should smooth divQ, better handling of data gaps

In [ ]:
# ===== OLD CHECK DEM FOR ERRORS AND REPLACE SCRIPT (no longer needed with OGGM processing) =====
#         #Create buffer around glacier polygon
#         glac_geom_buff = gf.glac_geom.Buffer(debris_prms.buff_dist)
#         #This is False over glacier polygon surface, True elsewhere - can be applied directly
#         glac_geom_buff_mask = geolib.geom2mask(glac_geom_buff, ds_dict['ice_thick'])
        
#             # ds masks
#             ds_list_masked = [iolib.ds_getma(i) for i in ds_list]
#             dem1 = np.ma.masked_less_equal(ds_list_masked[0], 0)
#             dems_mask = dem1.mask
#             if verbose:
#                 print('list of datasets:', len(ds_list_masked), fn_dict.values())

#             #Combine to identify ~1 km buffer around glacier polygon over static rock
#             static_buffer_mask = np.ma.mask_or(~glac_shp_lyr_mask, glac_geom_buff_mask)
#             static_shp_lyr_mask = np.ma.mask_or(static_buffer_mask, dems_mask)
        
        
#             # Check if DEM has huge errors or not - replace if necessary
#             if input.roi in ['01']:

#                 gf.z1_check = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)
#                 if gf.z1_check.min() < 0:

#                     # Add backup DEM for regions with known poor quality (ex. Alaska)
#                     print('switching DEMs')
#                     fn_dict['z1_backup'] = input.z1_backup_dict[input.roi]
#                     # Warp everything to common res/extent/proj (a second time)
#                     ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, \
#                             extent=warp_extent, t_srs=aea_srs, verbose=verbose, \
#                             r='cubic')
#                     ds_dict = dict(zip(fn_dict.keys(), ds_list))

#                     if verbose:
#                         print(ds_list)
#                         print(fn_dict.keys())

#                     # ds masks
#                     ds_list_masked = [iolib.ds_getma(i) for i in ds_list]
#                     dem1 = np.ma.masked_less_equal(ds_list_masked[-1], 0)
#                     dems_mask = dem1.mask
#                     if verbose:
#                         print('list of datasets:', len(ds_list_masked), fn_dict.values())

#                     #Combine to identify ~1 km buffer around glacier polygon over static rock
#                     static_buffer_mask = np.ma.mask_or(~glac_shp_lyr_mask, glac_geom_buff_mask)
#                     static_shp_lyr_mask = np.ma.mask_or(static_buffer_mask, dems_mask)

#                     #This is False over glacier polygon surface, True elsewhere - can be applied directly
#                     glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1_backup'])
#                     gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1_backup']), mask=glac_geom_mask)
#                     #gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)

#                     # Debris cover
#                     dc_mask = np.ma.mask_or(dc_shp_lyr_mask, glac_geom_mask)
#                     gf.dc_area = np.ma.array(iolib.ds_getma(ds_dict['z1_backup']), mask=dc_mask)